In [8]:
import pygame
import sys
import pandas as pd
import chess.pgn
import chess
import time
import numpy as np
from DeepChessEngine import DeepChessEngine

In [9]:
DCE = DeepChessEngine(depth = 3)

In [10]:
def board_to_numpy(board):
    numpy_board = [
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""]
]

    piece_to_int = {
        chess.PAWN: "p",
        chess.KNIGHT: "n",
        chess.BISHOP: 'b',
        chess.ROOK: 'r',
        chess.QUEEN: 'q',
        chess.KING: 'k'
    }


    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
        
            piece_color = piece.color
            #print(piece_color)
            piece_type = piece.piece_type

     
            if piece_color:
                piece_code = piece_to_int[piece_type].upper()
            else:
                piece_code = piece_to_int[piece_type].lower()
            
            rank, file = chess.square_rank(square),7 - chess.square_file(square)
            numpy_board[rank][file] = piece_code

    return numpy_board


In [11]:
board = chess.Board()
board.push_san('e4')
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [12]:
def get_position(coord):
    for i in range(8):
        if (800-coord[0])//100 == i:
            break
    for j in range(8):
        if coord[1]//100 == j:
            break
    d = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
    return d[i] + str(j + 1)


In [13]:
def selectSide():
    x = input("Write the side you want(w for white and b for black)")
    x = x.lower()
    if x=='w':
        return True
    elif x=='b':
        return False
    elif x!='w' or x!='b':
        return selectSide()

In [14]:
board = chess.Board()
prompt_message = ''
prompt_timestamp = 0

WIDTH, HEIGHT = 800, 800
ROWS, COLS = 8, 8
SQUARE_SIZE = WIDTH // COLS
clock=pygame.time.Clock()

WHITE = (240, 217, 181)
BLACK = (181, 136, 99)
GRAY = (128, 128, 128)


pygame.init()
window = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Chess")

chess_board = [
    ["R", "N", "B", "K", "Q", "B", "N", "R"],
    ["P", "P", "P", "P", "P", "P", "P", "P"],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["", "", "", "", "", "", "", ""],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    ["r", "n", "b", "k", "q", "b", "n", "r"]
]


def show_prompt(message):
    global prompt_message, prompt_timestamp
    prompt_message = message
    prompt_timestamp = pygame.time.get_ticks()
    
def draw_board(selected_cell):
    for row in range(ROWS):
        for col in range(COLS):
            color = WHITE if (row + col) % 2 == 0 else BLACK
            pygame.draw.rect(window, color, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
            
            if selected_cell is not None and (col, row) == selected_cell:
                pygame.draw.rect(window, (0, 0, 0), pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 3)
    selected_cell = None
    global prompt_message, prompt_timestamp
    if prompt_message is not None:
        elapsed_time = pygame.time.get_ticks() - prompt_timestamp
        if elapsed_time < 2000:  # Show prompt for 2 seconds (2000 milliseconds)
            font = pygame.font.SysFont(None, 30)
            text = font.render(prompt_message, True, (0, 0, 0))
            window.blit(text, (WIDTH // 4, HEIGHT//4))
        else:
            prompt_message = ''



def draw_pieces(chess_board):
    for col in range(ROWS):
        for row in range(COLS):
            piece = chess_board[row][col]
            if piece != "" :
                if piece.lower()!=piece:
                    piece_image = pygame.image.load(f"images/white/{piece}.png")
                    piece_image = pygame.transform.scale(piece_image,(90,90))
                    window.blit(piece_image, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
                else:
                    piece_image = pygame.image.load(f"images/black/{piece}.png")
                    piece_image = pygame.transform.scale(piece_image,(90,90))
                    window.blit(piece_image, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))
                    


def main(chess_board):
    board = chess.Board()
    chess_board = board_to_numpy(board)
    selected_cell = None
    a = ""
    b = ""
    c = 'q'
    i = 0
    k=1
    d = 1
    #d = int(input("Please Input depth of the engine-"))
    user_side = selectSide()
    while True:
        window.fill(GRAY)
        draw_board(selected_cell)
        draw_pieces(chess_board)
        pygame.display.update()
        clock.tick(60)
        if user_side == True:
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()    
                if event.type == pygame.MOUSEBUTTONDOWN and i==1:
                    (x2,y2)=pygame.mouse.get_pos()
                    
                    b=get_position((x2,y2))
                    i=2
                    selected_cell = None
                if event.type == pygame.MOUSEBUTTONDOWN and i==0:
                    (x1,y1)=pygame.mouse.get_pos()
                    
                    i=1
                    a=get_position((x1,y1))
                    selected_cell = (x1 // SQUARE_SIZE, y1 // SQUARE_SIZE)
            if a and b and i==2:
                x = a+b
                print(x)
                legal_moves = list(board.legal_moves)
                try:
                    move = chess.Move.from_uci(x)
                    if (b == 'a1' or 'b1' or 'c1' or 'd1' or 'e1' or 'f1' or 'g1' or 'h1' or 'a8' or 'b8' or 'c8' or 'd8' or 'e8' or 'f8' or 'g8' or 'h8') and (chess.Move.from_uci(a + b + c) in legal_moves):
                        c = input("What should the pawn be promoted to?(q,r,n,b)")
                        move = chess.Move.from_uci(a+b+c)
                        board.push(move)
                        if DCE.is_game_over(board):
                            show_prompt("Game Over")

                        i=0
                        chess_board=board_to_numpy(board)
                        
                        window.fill(GRAY)
                        draw_board(selected_cell)
                        draw_pieces(chess_board)
                        pygame.display.update()
                        clock.tick(60)
                        x = DCE.computerMove(board, d)
                        if DCE.is_game_over(board):
                            show_prompt("Game Over")
                        chess_board=board_to_numpy(board)
                        
                        window.fill(GRAY)
                        draw_board(selected_cell)
                        draw_pieces(chess_board)
                        pygame.display.update()
                        clock.tick(60)
                    
                    elif move in legal_moves:
                        board.push(move)
                        if DCE.is_game_over(board):
                            show_prompt("Game Over")

                        i=0
                    
                        chess_board=board_to_numpy(board)
                        
                        draw_board(selected_cell)
                        draw_pieces(chess_board)
                        pygame.display.update()
                        clock.tick(60)
                        x = DCE.computerMove(board, d)
                        if DCE.is_game_over(board):
                            show_prompt("Game Over")
                        chess_board=board_to_numpy(board)
                        
                        window.fill(GRAY)
                        draw_board(selected_cell)
                        draw_pieces(chess_board)
                        pygame.display.update()
                        clock.tick(60)
                    
                    else:
                        raise ValueError("Invalid move")
                except ValueError as e:
                    print(e)
                    i = 0
                    a = ""
                    b = ""
                    
                    continue
            
            if DCE.is_game_over(board):
                print('Game Over')
                pygame.quit()
                sys.exit()

        elif user_side == False:
            if k==1:
                chess_board=board_to_numpy(board)
                
                x = DCE.computerMove(board, d)
                if DCE.is_game_over(board):
                    show_prompt("Game Over")
                chess_board=board_to_numpy(board)
                
                draw_board(selected_cell)
                draw_pieces(chess_board)
                pygame.display.update()
                clock.tick(60)
   
                k+=1
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()    
                    if event.type == pygame.MOUSEBUTTONDOWN and i==1:
                        (x2,y2)=pygame.mouse.get_pos()
                        
                        b=get_position((x2,y2))
                        i=2
                        selected_cell = None
                    if event.type == pygame.MOUSEBUTTONDOWN and i==0:
                        (x1,y1)=pygame.mouse.get_pos()
                       
                        i=1
                        a=get_position((x1,y1))
                        selected_cell = (x1 // SQUARE_SIZE, y1 // SQUARE_SIZE)
                if a and b and i==2:
                    x = a+b
                    print(x)
                    legal_moves = list(board.legal_moves)
                    try:
                        move = chess.Move.from_uci(x)
                        if (b == 'a1' or 'b1' or 'c1' or 'd1' or 'e1' or 'f1' or 'g1' or 'h1' or 'a8' or 'b8' or 'c8' or 'd8' or 'e8' or 'f8' or 'g8' or 'h8') and (chess.Move.from_uci(a + b + c) in legal_moves):
                            c = input("What should the pawn be promoted to?(q,r,n,b)")
                            move = chess.Move.from_uci(a+b+c)
                            board.push(move)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
    
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                            i=0
                            chess_board=board_to_numpy(board)
                            
                            x = DCE.computerMove(board, d)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)

                        elif move in legal_moves:
                            board.push(move)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
    
                            i=0
                   
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                            x = DCE.computerMove(board, d)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                       
                        else:
                            raise ValueError("Invalid move")
                    except ValueError as e:
                        print(e)
                        i = 0
                        a = ""
                        b = ""
                        
                        continue
        
                    

            else:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()    
                    if event.type == pygame.MOUSEBUTTONDOWN and i==1:
                        (x2,y2)=pygame.mouse.get_pos()
                        
                        b=get_position((x2,y2))
                        i=2
                        selected_cell = None
                    if event.type == pygame.MOUSEBUTTONDOWN and i==0:
                        (x1,y1)=pygame.mouse.get_pos()
                       
                        i=1
                        a=get_position((x1,y1))
                        selected_cell = (x1 // SQUARE_SIZE, y1 // SQUARE_SIZE)
                if a and b and i==2:
                    x = a+b
                    print(x)
                    legal_moves = list(board.legal_moves)
                    try:
                        move = chess.Move.from_uci(x)
                        if (b == 'a1' or 'b1' or 'c1' or 'd1' or 'e1' or 'f1' or 'g1' or 'h1' or 'a8' or 'b8' or 'c8' or 'd8' or 'e8' or 'f8' or 'g8' or 'h8') and (chess.Move.from_uci(a + b + c) in legal_moves):
                            c = input("What should the pawn be promoted to?(q,r,n,b)")
                            move = chess.Move.from_uci(a+b+c)
                            board.push(move)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
    
                            i=0
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                            x = DCE.computerMove(board, d)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                        elif move in legal_moves:
                            board.push(move)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
    
                            i=0
                
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                            x = DCE.computerMove(board, d)
                            if DCE.is_game_over(board):
                                show_prompt("Game Over")
                            chess_board=board_to_numpy(board)
                            
                            draw_board(selected_cell)
                            draw_pieces(chess_board)
                            pygame.display.update()
                            clock.tick(60)
                        else:
                            raise ValueError("Invalid move")
                    except ValueError as e:
                        print(e)
                        i = 0
                        a = ""
                        b = ""
                        
                        continue
                    
            

if __name__ == "__main__":
    main(chess_board)


(445, 150)
(445, 244)
d2d3
1/1 [==============================] - 0s 9ms/step
(354, 156)
(356, 260)
e2e3
1/1 [==============================] - 0s 10ms/step
(550, 146)
(550, 231)
c2c3
1/1 [==============================] - 0s 10ms/step
(550, 246)
(550, 315)
c3c4
1/1 [==============================] - 0s 10ms/step
(543, 333)
(543, 461)
c4c5
1/1 [==============================] - 0s 10ms/step
(434, 246)
(444, 314)
d3d4
1/1 [==============================] - 0s 10ms/step
(445, 47)
(759, 349)
d1a4
1/1 [==============================] - 0s 10ms/step
(759, 349)
(455, 649)
a4d7
1/1 [==============================] - 0s 10ms/step
(331, 72)
(465, 159)
e1d2
1/1 [==============================] - 0s 10ms/step
(654, 25)
(564, 220)
b1c3
1/1 [==============================] - 0s 9ms/step


SystemExit: 

/Users/akshaankhan2004/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
